[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tulasiram58827/craft_tflite/blob/main/colabs/savedmodel_to_tflite.ipynb)

This notebook converts tensorflow savel model to tflite version

## SetUp

In [1]:
"""
Authors
 * Tulasi Ram
"""
!pip install tf-nightly
!pip install onnx
!pip install git+https://github.com/onnx/onnx-tensorflow.git

import onnx

import tensorflow as tf
from onnx_tf.backend import prepare

     |████████████████████████████████| 397.2 MB 25 kB/s  eta 0:00:01    |████                            | 48.9 MB 1.9 MB/s eta 0:03:07
     |████████████████████████████████| 14.5 MB 3.0 MB/s eta 0:00:01
  Using cached wheel-0.35.1-py2.py3-none-any.whl (33 kB)
  Using cached wrapt-1.12.1.tar.gz (27 kB)
     |████████████████████████████████| 10.6 MB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 461 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 346 kB/s eta 0:00:01
     |████████████████████████████████| 127 kB 4.2 MB/s eta 0:00:01
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-linux_x86_64.whl size=78025 sha256=fe14df0a4172be5d4c6d672212f8b98e30d983f22aa80db6e6303263b792b54a
  Stored in directory: /home/ram/.cache/pip/wheels/5f/fd/9e/b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
Successfully built wrapt
  Attempting uninstall: numpy
    Found existing installat

  Cloning https://github.com/onnx/onnx-tensorflow.git to /tmp/pip-req-build-gu_tmg4l
  Created wheel for onnx-tf: filename=onnx_tf-1.6.0-py3-none-any.whl size=205527 sha256=e276a1179d27938f4ebebca14106eefa04eb0221152bb10f9643d34f38e82838
  Stored in directory: /tmp/pip-ephem-wheel-cache-csd7zidx/wheels/e9/b7/67/8abcf50ca14ef4375302d30abf9f4242e06be09b829f5fd307
Successfully built onnx-tf


/home/ram/anaconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:38: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20201117). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


### Export to tensorflow graph

In [10]:
# Export model to tensorflow

# craft.onnx will be available after running this pytorch_to_onnx.ipynb

onnx_model = onnx.load('../models/craft.onnx')
tf_rep = prepare(onnx_model)
tf_rep.export_graph('../models/craft.pb')

print("Model converted to tensorflow graph succesfully.")

INFO:tensorflow:Assets written to: ../models/craft.pb/assets


INFO:tensorflow:Assets written to: ../models/craft.pb/assets


Model converted to tensorflow graph succesfully.


### Conversion to TFLite

In [11]:
# Download dataset from here https://drive.google.com/file/d/1HvujMMNCG49esPpZOHEnPFY6k-khbrmZ/view?usp=sharing

dataset_path = '/home/ram/Projects/OCR/test_files/'


loaded = tf.saved_model.load('../models/craft.pb')

concrete_func = loaded.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

concrete_func.inputs[0].set_shape([None, 3, 320, 200])

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])

converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [12]:
# Uncomment this line for float16 quantization.
# converter.target_spec.supported_types = [tf.float16]
# converter.target_spec.supported_types = [tf.lite.OpsSet.TFLITE_BUILTINS]


# Uncomment For Integer Quantization
# def representative_data_gen():
#     for file in os.listdir(dataset_path)[:10]:
#         file_path = dataset_path+file
#         image = imgproc.loadImage(file_path)
#         image = cv2.resize(image, dsize=(800, 1280), interpolation=cv2.INTER_LINEAR)
#         img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(image, 1280, interpolation=cv2.INTER_LINEAR, mag_ratio=1.5)
#         ratio_h = ratio_w = 1 / target_ratio

#         # preprocessing
#         x = imgproc.normalizeMeanVariance(img_resized)
#         x = torch.from_numpy(x).permute(2, 0, 1)    # [h, w, c] to [c, h, w]
#         x = Variable(x.unsqueeze(0))                # [c, h, w] to [b, c, h, w]
#         x = x.cpu().detach().numpy()
#         yield [x]

# converter.representative_dataset = representative_data_gen

tf_lite_model = converter.convert()

INFO:absl:Using new converter: If you encounter a problem please file a bug. You can opt-out by setting experimental_new_converter=False


ConverterError: /home/ram/anaconda3/lib/python3.8/site-packages/tensorflow/python/saved_model/load.py:890:0: error: 'tf.MaxPool' op is neither a custom op nor a flex op
/home/ram/anaconda3/lib/python3.8/site-packages/tensorflow/python/saved_model/load.py:860:0: note: called from
<ipython-input-11-cbb577ab5e2e>:6:0: note: called from
/home/ram/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343:0: note: called from
/home/ram/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263:0: note: called from
/home/ram/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071:0: note: called from
/home/ram/anaconda3/lib/python3.8/site-packages/IPython/core/async_helpers.py:68:0: note: called from
/home/ram/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2895:0: note: called from
/home/ram/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2866:0: note: called from
/home/ram/anaconda3/lib/pyth

In [ ]:
open('craft.tflite', 'wb').write(tf_lite_model)

print("Converted to tensorflow lite succesfully.")